<div style="background-color: #fde70e; padding: 10px; display: flex; align-items: center; border-bottom: 2px solid #fde70e;">
  <a href="https://www.fhnw.ch/en/about-fhnw/schools/school-of-engineering/institutes/institute-of-electric-power-systems" target="_blank">
    <img src="./data/Logo_FHNW.png" alt="FHNW Logo" style="height: 40px; margin-right: 15px;">
  </a>
</div>

# Energy Consumption Calculator for Drive Systems
This tool calculates the energy consumption of a drive system based on a load profile $P_{out}(t)$ and efficiency data $\eta(P_{out})$, both provided by the user.  
When providing efficiency data of a second drive system, the energy consumptions and costs of both systems can be compared.  

<div style="background-color: #f5f5f5; padding: 10px; border-left: 4px solid #ccc; margin-top: 15px; margin-bottom: 30px;">
  <p style="margin: 0;">
    <strong>Developed by:</strong><br>
    <a href="https://www.fhnw.ch/en/about-fhnw/schools/school-of-engineering/institutes/institute-of-electric-power-systems" target="_blank">
      Institute of Electric Power Systems, FHNW
    </a><br>
    <strong>Contact:</strong><br>
    <a href="mailto:nicola.schulz@fhnw.ch">Nicola Schulz</a><br>
    <strong>Source code:</strong><br>
    <a href="https://github.com/nicolaschulz/drive-efficiency-tool" target="_blank">
      github.com/nicolaschulz/drive-efficiency-tool
    </a>
  </p>
</div>

<details>
  <summary style="font-size: 1.1em; font-weight: bold; cursor: pointer;">📂 Instructions for Uploading Data Files</summary>
  <p>Upload Excel files with the following contents:</p>
  <ol>
    <li><strong>Efficiency data related to the output power:</strong> Column A = "Power", column B = "Efficiency" (0...1).</li>
    <li><strong>Load profile of the drive system:</strong> Column A = "Power", each row = one time step.<br> <b>Important:</b> The load profile must not contain any power values outside the range covered by the efficiency profile!<br>Power values of zero are allowed.</li>
    <li><strong>Optional:</strong> A second efficiency data file for comparing two drive systems for the same load profile.</li>
  </ol>

  <table>
    <tr>
      <td align="center">
        <img src="data/exceltemplate_eff_p.png" alt="Efficiency template" height="200"><br>
        <em>Fig. 1: Efficiency table example ("Power" = output power in W)<br>Efficiency values between 0 and 1.</em><br>
        <a href="https://github.com/nicolaschulz/drive-efficiency-tool/raw/main/data/example_efficiency.xlsx">📥 Download example</a>
      </td>
      <td style="width: 40px;"></td>
      <td align="center">
        <img src="data/exceltemplate_up_power.png" alt="Usage profile template" height="200"><br>
        <em>Fig. 2: Load profile example ("Power" = output power in W).<br>Column B is not required.</em><br>
        <a href="https://github.com/nicolaschulz/drive-efficiency-tool/raw/main/data/example_usage_profile_1.xlsx">📥 Download example</a>
      </td>
    </tr>
  </table>
</details>

<details>
  <summary style="font-size: 1.1em; font-weight: bold; cursor: pointer;">⚙️ Parameter Input Guide</summary>
  <div style="margin-top: 10px;">
    <ul>
      <li><strong>Max. Power:</strong> Maximum output power of the system. This value must not exceed the highest power value listed in the efficiency table. Specify in watts.</li>
      <li><strong>Price:</strong> Electricity price in CHF/kWh used to calculate energy costs.</li>
      <li><strong>Timestep:</strong> Time interval of one row in the Excel load profile file.</li>
      <li><strong>Time unit:</strong> Unit of the time step (seconds / minutes / hours)</li>
      <li><strong>Fit:</strong> For each uploaded efficiency dataset, different fit functions can be selected to interpolate between data points. The data points and fit functions will be displayed once the user clicks “Calculate”.</li>
    </ul>
  </div>
</details>

<details>
  <summary style="font-size: 1.1em; font-weight: bold; cursor: pointer;">📊 Tool Output</summary>
  <div style="margin-top: 10px;">
    <p>The following results are generated:</p>
    <ul>
      <li>Visualization of the efficiency data and selected fit functions</li>
      <li>Energy consumption, useful energy, losses, and energy cost for one run of the load profile</li>
      <li>Energy consumption, useful energy, losses, and energy cost extrapolated to one year</li>
      <li>Time series plots of input and output power</li>
    </ul>
  </div>
</details>

<div style="background-color: #fde70e; border-left: 5px solid #f0c000; padding: 15px 20px; margin-top: 30px; box-shadow: 2px 2px 5px rgba(0,0,0,0.1);">
  <h2 style="margin: 0;">🛠️ Tool Input Section</h2>
  <p style="margin: 5px 0 0;">This is the interactive part: define parameters, upload files, and start the calculation.</p>
</div>

In [ ]:

def handle_calculate(b):
    global usage_profile_df, efficiency_df
    output.clear_output()

    with output:
        power_in2 = None  # vorbereiten für optionalen zweiten Plot

        if not upload_usage_profile.value or not upload_efficiency_data.value:
            print("⚠️ Please upload Excel files for profile and efficiency 1.")
            return

        try:
            usage_data = upload_usage_profile.value[0]
            efficiency_data = upload_efficiency_data.value[0]

            usage_name = usage_data['name']
            eff_name = efficiency_data['name']

            usage_profile_df = pd.read_excel(io.BytesIO(usage_data['content']))
            efficiency_df = pd.read_excel(io.BytesIO(efficiency_data['content']))

            power_out = usage_profile_df['Power'].tolist()

            # Fit 1 + Berechnung
            efficiency = fit_efficiency_curve(efficiency_df, fit_type_field.value)
            power_in = [p / efficiency(p) for p in power_out]
            ein = Integrate(power_in, timestep_field.value, unit=energy_unit_field.value)
            eout = Integrate(power_out, timestep_field.value, unit=energy_unit_field.value)
            edelta = ein - eout
            ecost = ein * energy_price_field.value

            print(f"✅ Used files:")
            print(f"📂 Usage Profile: {usage_name}")
            print(f"📂 Efficiency Data 1: {eff_name}")

            plot_efficiency_fit(efficiency_df, fit_type_field.value)

            print("🔋 Energy consumption (P_in 1): {:.3f} kWh".format(ein))
            print("⚙️ Energy used for application (P_out): {:.3f} kWh".format(eout))
            print("🔥 Energy losses: {:.3f} kWh".format(edelta))
            print("💰 Energy cost: {:.2f} CHF".format(ecost))

            # 🔁 Dauer des Profils (in Sekunden) für Hochrechnung
            timestep = timestep_field.value
            unit = energy_unit_field.value
            duration_s = len(power_out) * timestep
            if unit == 'minutes':
                duration_s *= 60
            elif unit == 'hours':
                duration_s *= 3600

            cycles_per_year = (365 * 24 * 3600) / duration_s
            ein_year = ein * cycles_per_year
            edelta_year = edelta * cycles_per_year
            ecost_year = ecost * cycles_per_year

            print("\n📆 Extrapolation to 1 year (efficiency 1):")
            print("🔋 Annual consumption: {:.0f} kWh".format(ein_year))
            print("🔥 Annual energetic losses: {:.0f} kWh".format(edelta_year))
            print("💰 Annual energy cost: {:.0f} CHF".format(ecost_year))

            if use_second_eff_field.value and upload_efficiency_data_2.value:
                try:
                    eff2_data = upload_efficiency_data_2.value[0]
                    eff2_name = eff2_data['name']
                    eff2_df = pd.read_excel(io.BytesIO(eff2_data['content']))

                    print(f"\n📂 Efficiency Data 2: {eff2_name}")
                    plot_efficiency_fit(eff2_df, fit_type_field_2.value)

                    efficiency2 = fit_efficiency_curve(eff2_df, fit_type_field_2.value)
                    power_in2 = [p / efficiency2(p) for p in power_out]

                    ein2 = Integrate(power_in2, timestep_field.value, unit=energy_unit_field.value)
                    edelta2 = ein2 - eout
                    ecost2 = ein2 * energy_price_field.value

                    print("🔋 Energy consumption (P_in 2): {:.3f} kWh".format(ein2))
                    print("🔥 Energy losses: {:.3f} kWh".format(edelta2))
                    print("💰 Energy cost: {:.2f} CHF".format(ecost2))

                    print("\n📊 Comparison of the consumption data for the given load profile for the two efficiency curves:")
                    compare_results(ein, ecost, edelta, ein2, ecost2, edelta2)

                    compare_yearly_results(ein, ecost, edelta, ein2, ecost2, edelta2, duration_s)

                except Exception as e2:
                    print("❌ Error in second efficiency calculation:", e2)

            plot_power_curves(
                power_out,
                power_in,
                timestep_field.value,
                energy_unit_field.value,
                power_in2
            )

        except Exception as e:
            print("❌ Error in the calculation:", e)


In [ ]:
import matplotlib.pyplot as plt

def plot_power_curves(power_out, power_in1, timestep, unit, power_in2=None):
    if unit == 'minutes':
        dt = timestep * 60
    elif unit == 'hours':
        dt = timestep * 3600
    else:
        dt = timestep

    t = [i * dt for i in range(len(power_out))]

    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(t, power_out, label='P_out (Load profile)', linewidth=2)
    ax.plot(t, power_in1, label='P_in 1 (Efficiency 1)', linewidth=2, linestyle='--')

    if power_in2 is not None:
        ax.plot(t, power_in2, label='P_in 2 (Efficiency 2)', linewidth=2, linestyle=':', color='green')

    ax.set_xlabel('Time [s]')
    ax.set_ylabel('Power [W]')
    ax.set_title('P_out, P_in 1 and P_in 2 (if applicable)')
    ax.grid(True)
    ax.legend()
    plt.show()
